In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Ali Sobhani Thesis/Noisy/TS'


/content/drive/.shortcut-targets-by-id/10ul3ZK_MC4nZuEgYotLTtUO0eEZB3gue/Ali Sobhani Thesis/Noisy/TS


In [3]:
!pip install scikit-optimize
!pip install scikeras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.3 MB/s eta 0:00:00


In [4]:
from tensorflow.keras.layers import Flatten, Conv2D, DepthwiseConv2D, BatchNormalization, Dense, Activation, Dropout, SeparableConv2D
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.constraints import max_norm
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
import numpy as np


############################
X = np.load('X.npy', allow_pickle=True)
Y = np.load('Y.npy')
X_20 = np.load('X_TS_SNR_20.npy', allow_pickle=True)
Y_20 = np.load('Y_TS_SNR_20.npy')
X_25 = np.load('X_TS_SNR_25.npy', allow_pickle=True)
Y_25 = np.load('Y_TS_SNR_25.npy')
X_30 = np.load('X_TS_SNR_30.npy', allow_pickle=True)
Y_30 = np.load('Y_TS_SNR_30.npy')
X_35 = np.load('X_TS_SNR_35.npy', allow_pickle=True)
Y_35 = np.load('Y_TS_SNR_35.npy')
X_40 = np.load('X_TS_SNR_40.npy', allow_pickle=True)
Y_40 = np.load('Y_TS_SNR_40.npy')
X_45 = np.load('X_TS_SNR_45.npy', allow_pickle=True)
Y_45 = np.load('Y_TS_SNR_45.npy')
###########################

def create_model3(optimizer='adam', kernel_size=3, dropout=0.5, norm_rate=0.01, F1=16, D=2, activation='relu', kernelsize =2, F2=32):
    input1       = Input(shape = (8, 7, 1))
    block1       = Conv2D(F1, (1, kernel_size), padding = 'same',
                                    input_shape = (1, 8, 7),
                                    use_bias = False)(input1)
    block1       = BatchNormalization(axis = -1)(block1)
    block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                      depth_multiplier = D,
                                      depthwise_constraint = max_norm(norm_rate))(block1)
    block2       = BatchNormalization(axis = -1)(block2)
    if activation == 'leakyrelu':
        block2   = LeakyReLU()(block2)
    else:
        block2   = Activation(activation)(block2)
    block3       = Dropout(dropout)(block2)
    block4       = SeparableConv2D(F2, (1, kernelsize),
                                    use_bias = False, padding = 'same')(block3)
    block5       = BatchNormalization(axis = -1)(block4)
    if activation == 'leakyrelu':
        block6   = LeakyReLU()(block5)
    else:
        block6   = Activation(activation)(block5)
    flatten      = Flatten(name = 'flatten')(block6)
    block7       = Dropout(dropout)(flatten)
    dense        = Dense(1, name = 'dense',
                          kernel_constraint = max_norm(norm_rate))(block7)
    sigmoid      = Activation('sigmoid', name = 'sigmoid')(dense)
    model        = Model(inputs=input1, outputs=sigmoid)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

p = {
    'optimizer': ['Adam', 'Adamax', 'Nadam'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 15, 20, 30],
    'kernel_size': [2, 3, 4, 5],
    'dropout': [0.2, 0.3, 0.4, 0.5],
    'norm_rate': [0.01, 0.001, 0.1, 1, 5, 10],
    'F1': [16, 32, 64],
    'D': [1, 2, 3],
    'activation': ['relu', 'elu', 'leakyrelu'],
    'F2' : [32,64,128],
    'kernelsize' : [2,3]
}

model = KerasClassifier(model=create_model3, verbose=1, **p)
inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=13)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)

bayes = BayesSearchCV(estimator=model, search_spaces=p, n_iter=10, n_points=2, cv=inner_cv, scoring='accuracy')

from sklearn.metrics import f1_score, accuracy_score
best_scores = {'accuracy': [], 'f2': [], 'f1':[]}
best_scores_20 = {'accuracy': [], 'f2': [], 'f1':[]}
best_scores_25 = {'accuracy': [], 'f2': [], 'f1':[]}
best_scores_30 = {'accuracy': [], 'f2': [], 'f1':[]}
best_scores_35 = {'accuracy': [], 'f2': [], 'f1':[]}
best_scores_40 = {'accuracy': [], 'f2': [], 'f1':[]}
best_scores_45 = {'accuracy': [], 'f2': [], 'f1':[]}

for train_idx, test_idx in outer_cv.split(X, Y):
    X_train, X_test, X_test_20, X_test_25, X_test_30, X_test_35, X_test_40, X_test_45 = X[train_idx], X[test_idx], X_20[test_idx], X_25[test_idx], X_30[test_idx], X_35[test_idx], X_40[test_idx], X_45[test_idx]
    y_train, y_test, y_test_20, y_test_25, y_test_30, y_test_35, y_test_40, y_test_45 = Y[train_idx], Y[test_idx], Y_20[test_idx], Y_25[test_idx], Y_30[test_idx], Y_35[test_idx], Y_40[test_idx], Y_45[test_idx]

    bayes.fit(X_train, y_train)

    y_pred = bayes.predict(X_test)
    y_pred_20 = bayes.predict(X_test_20)
    y_pred_25 = bayes.predict(X_test_25)
    y_pred_30 = bayes.predict(X_test_30)
    y_pred_35 = bayes.predict(X_test_35)
    y_pred_40 = bayes.predict(X_test_40)
    y_pred_45 = bayes.predict(X_test_45)

    f2 = fbeta_score(y_test, y_pred, beta=2)
    f2_20 = fbeta_score(y_test_20, y_pred_20, beta=2)
    f2_25 = fbeta_score(y_test_25, y_pred_25, beta=2)
    f2_30 = fbeta_score(y_test_30, y_pred_30, beta=2)
    f2_35 = fbeta_score(y_test_35, y_pred_35, beta=2)
    f2_40 = fbeta_score(y_test_40, y_pred_40, beta=2)
    f2_45 = fbeta_score(y_test_45, y_pred_45, beta=2)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_20 = accuracy_score(y_test_20, y_pred_20)
    accuracy_25 = accuracy_score(y_test_25, y_pred_25)
    accuracy_30 = accuracy_score(y_test_30, y_pred_30)
    accuracy_35 = accuracy_score(y_test_35, y_pred_35)
    accuracy_40 = accuracy_score(y_test_40, y_pred_40)
    accuracy_45 = accuracy_score(y_test_45, y_pred_45)

    f1 = f1_score(y_test, y_pred)
    f1_20 = f1_score(y_test_20, y_pred_20)
    f1_25 = f1_score(y_test_25, y_pred_25)
    f1_30 = f1_score(y_test_30, y_pred_30)
    f1_35 = f1_score(y_test_35, y_pred_35)
    f1_40 = f1_score(y_test_40, y_pred_40)
    f1_45 = f1_score(y_test_45, y_pred_45)

    best_scores['f2'].append(f2)
    best_scores_20['f2'].append(f2_20)
    best_scores_25['f2'].append(f2_25)
    best_scores_30['f2'].append(f2_30)
    best_scores_35['f2'].append(f2_35)
    best_scores_40['f2'].append(f2_40)
    best_scores_45['f2'].append(f2_45)

    best_scores['f1'].append(f1)
    best_scores_20['f1'].append(f1_20)
    best_scores_25['f1'].append(f1_25)
    best_scores_30['f1'].append(f1_30)
    best_scores_35['f1'].append(f1_35)
    best_scores_40['f1'].append(f1_40)
    best_scores_45['f1'].append(f1_45)

    best_scores['accuracy'].append(accuracy)
    best_scores_20['accuracy'].append(accuracy_20)
    best_scores_25['accuracy'].append(accuracy_25)
    best_scores_30['accuracy'].append(accuracy_30)
    best_scores_35['accuracy'].append(accuracy_35)
    best_scores_40['accuracy'].append(accuracy_40)
    best_scores_45['accuracy'].append(accuracy_45)


Epoch 1/15
9/9 [==============================] - 14s 20ms/step - loss: 0.5030 - accuracy: 0.7298
Epoch 2/15
9/9 [==============================] - 0s 5ms/step - loss: 0.2860 - accuracy: 0.8750
Epoch 3/15
9/9 [==============================] - 0s 5ms/step - loss: 0.2170 - accuracy: 0.9231
Epoch 4/15
9/9 [==============================] - 0s 5ms/step - loss: 0.1791 - accuracy: 0.9279
Epoch 5/15
9/9 [==============================] - 0s 5ms/step - loss: 0.1450 - accuracy: 0.9510
Epoch 6/15
9/9 [==============================] - 0s 5ms/step - loss: 0.1290 - accuracy: 0.9606
Epoch 7/15
9/9 [==============================] - 0s 5ms/step - loss: 0.1153 - accuracy: 0.9654
Epoch 8/15
9/9 [==============================] - 0s 5ms/step - loss: 0.1002 - accuracy: 0.9692
Epoch 9/15
9/9 [==============================] - 0s 5ms/step - loss: 0.0910 - accuracy: 0.9663
Epoch 10/15
9/9 [==============================] - 0s 5ms/step - loss: 0.0755 - accuracy: 0.9769
Epoch 11/15
9/9 [====================

In [5]:
average_best_scores = {scoring: np.mean(scores) for scoring, scores in best_scores.items()}
average_best_scores_20 = {scoring: np.mean(scores) for scoring, scores in best_scores_20.items()}
average_best_scores_25 = {scoring: np.mean(scores) for scoring, scores in best_scores_25.items()}
average_best_scores_30 = {scoring: np.mean(scores) for scoring, scores in best_scores_30.items()}
average_best_scores_35 = {scoring: np.mean(scores) for scoring, scores in best_scores_35.items()}
average_best_scores_40 = {scoring: np.mean(scores) for scoring, scores in best_scores_40.items()}
average_best_scores_45 = {scoring: np.mean(scores) for scoring, scores in best_scores_45.items()}

print(f"Average best scores: {average_best_scores}")
print(f"Average best scores_20: {average_best_scores_20}")
print(f"Average best scores_25: {average_best_scores_25}")
print(f"Average best scores_30: {average_best_scores_30}")
print(f"Average best scores_35: {average_best_scores_35}")
print(f"Average best scores_40: {average_best_scores_40}")
print(f"Average best scores_45: {average_best_scores_45}")

Average best scores: {'accuracy': 0.9550000000000001, 'f2': 0.8703014539148993, 'f1': 0.8937723496726633}
Average best scores_20: {'accuracy': 0.9496183375166101, 'f2': 0.8561341921764838, 'f1': 0.8836171979242327}
Average best scores_25: {'accuracy': 0.9538468920714601, 'f2': 0.8684654825261109, 'f1': 0.8927453640886476}
Average best scores_30: {'accuracy': 0.9557692307692307, 'f2': 0.8725497826426791, 'f1': 0.8957630357630357}
Average best scores_35: {'accuracy': 0.9557692307692307, 'f2': 0.8722222222222221, 'f1': 0.8949771689497716}
Average best scores_40: {'accuracy': 0.9553846153846154, 'f2': 0.8712629896083133, 'f1': 0.894376568349171}
Average best scores_45: {'accuracy': 0.9557692307692307, 'f2': 0.8722222222222221, 'f1': 0.8949771689497716}


In [6]:
import pandas as pd
import openpyxl
model = ['Model1', 'Model2', 'Model3', 'Model4']
FS = ['Bayes', 'Random', 'Grid']
# Load the existing file
book = openpyxl.load_workbook('TS_Results.xlsx')
book_20 = openpyxl.load_workbook('TS_Results_20.xlsx')
book_25 = openpyxl.load_workbook('TS_Results_25.xlsx')
book_30 = openpyxl.load_workbook('TS_Results_30.xlsx')
book_35 = openpyxl.load_workbook('TS_Results_35.xlsx')
book_40 = openpyxl.load_workbook('TS_Results_40.xlsx')
book_45 = openpyxl.load_workbook('TS_Results_45.xlsx')

# Prepare the data to be written
data_acc = average_best_scores['accuracy']
data_acc_20 = average_best_scores_20['accuracy']
data_acc_25 = average_best_scores_25['accuracy']
data_acc_30 = average_best_scores_30['accuracy']
data_acc_35 = average_best_scores_35['accuracy']
data_acc_40 = average_best_scores_40['accuracy']
data_acc_45 = average_best_scores_45['accuracy']

data_f2 = average_best_scores['f2']
data_f2_20 = average_best_scores_20['f2']
data_f2_25 = average_best_scores_25['f2']
data_f2_30 = average_best_scores_30['f2']
data_f2_35 = average_best_scores_35['f2']
data_f2_40 = average_best_scores_40['f2']
data_f2_45 = average_best_scores_45['f2']

data_f1 = average_best_scores['f1']
data_f1_20 = average_best_scores_20['f1']
data_f1_25 = average_best_scores_25['f1']
data_f1_30 = average_best_scores_30['f1']
data_f1_35 = average_best_scores_35['f1']
data_f1_40 = average_best_scores_40['f1']
data_f1_45 = average_best_scores_45['f1']
# Get the existing sheets
sheet_acc = book['ACC']
sheet_acc_20 = book_20['ACC']
sheet_acc_25 = book_25['ACC']
sheet_acc_30 = book_30['ACC']
sheet_acc_35 = book_35['ACC']
sheet_acc_40 = book_40['ACC']
sheet_acc_45 = book_45['ACC']

sheet_f2 = book['F2']
sheet_f2_20 = book_20['F2']
sheet_f2_25 = book_25['F2']
sheet_f2_30 = book_30['F2']
sheet_f2_35 = book_35['F2']
sheet_f2_40 = book_40['F2']
sheet_f2_45 = book_45['F2']

sheet_f1 = book['F1']
sheet_f1_20 = book_20['F1']
sheet_f1_25 = book_25['F1']
sheet_f1_30 = book_30['F1']
sheet_f1_35 = book_35['F1']
sheet_f1_40 = book_40['F1']
sheet_f1_45 = book_45['F1']
# Calculate the correct row and column numbers
row = model.index('Model3') + 2  # +2 because Excel index starts from 1 and row 1 contains headers
col = FS.index('Bayes') + 2  # +2 because Excel index starts from 1 and column 1 contains headers

# Write to the ACC sheet
sheet_acc.cell(row=row, column=col, value=data_acc)
sheet_acc_20.cell(row=row, column=col, value=data_acc_20)
sheet_acc_25.cell(row=row, column=col, value=data_acc_25)
sheet_acc_30.cell(row=row, column=col, value=data_acc_30)
sheet_acc_35.cell(row=row, column=col, value=data_acc_35)
sheet_acc_40.cell(row=row, column=col, value=data_acc_40)
sheet_acc_45.cell(row=row, column=col, value=data_acc_45)

# Write to the F2 sheet
sheet_f2.cell(row=row, column=col, value=data_f2)
sheet_f2_20.cell(row=row, column=col, value=data_f2_20)
sheet_f2_25.cell(row=row, column=col, value=data_f2_25)
sheet_f2_30.cell(row=row, column=col, value=data_f2_30)
sheet_f2_35.cell(row=row, column=col, value=data_f2_35)
sheet_f2_40.cell(row=row, column=col, value=data_f2_40)
sheet_f2_45.cell(row=row, column=col, value=data_f2_45)

# Write to the F1 sheet
sheet_f1.cell(row=row, column=col, value=data_f1)
sheet_f1_20.cell(row=row, column=col, value=data_f1_20)
sheet_f1_25.cell(row=row, column=col, value=data_f1_25)
sheet_f1_30.cell(row=row, column=col, value=data_f1_30)
sheet_f1_35.cell(row=row, column=col, value=data_f1_35)
sheet_f1_40.cell(row=row, column=col, value=data_f1_40)
sheet_f1_45.cell(row=row, column=col, value=data_f1_45)
# Save and close the Excel file
book.save('TS_Results.xlsx')
book_20.save('TS_Results_20.xlsx')
book_25.save('TS_Results_25.xlsx')
book_30.save('TS_Results_30.xlsx')
book_35.save('TS_Results_35.xlsx')
book_40.save('TS_Results_40.xlsx')
book_45.save('TS_Results_45.xlsx')